In [23]:
from nba_api.stats.endpoints import PlayerGameLog, ScoreboardV2, CommonTeamRoster
import pandas as pd
from nba_api.stats.static import players
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, Lasso, RidgeCV, Ridge, ElasticNetCV, ElasticNet, BayesianRidge, LogisticRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)

In [24]:

player_id = players.find_players_by_full_name("Kevin Durant")[0]["id"]
df = pd.concat([pd.DataFrame(PlayerGameLog(player_id=player_id, season="2023-24").get_data_frames()[0]), pd.DataFrame(PlayerGameLog(player_id=player_id, season="2022-23").get_data_frames()[0])])
df['HOME'] = df['MATCHUP'].apply(lambda x: 1 if 'vs. ' in x else 0)
df['OPPONENT'] = df['MATCHUP'].apply(lambda x: x.split()[-1])
df

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,HOME,OPPONENT
0,22023,201142,0022301194,"APR 14, 2024",PHX @ MIN,W,38,6,14,0.429,1,3,0.333,2,2,1.000,2,2,4,1,2,1,4,3,15,30,1,0,MIN
1,22023,201142,0022301185,"APR 12, 2024",PHX @ SAC,W,42,11,20,0.550,3,7,0.429,3,3,1.000,1,4,5,3,0,0,3,2,28,-4,1,0,SAC
2,22023,201142,0022301165,"APR 10, 2024",PHX @ LAC,W,37,10,21,0.476,2,9,0.222,2,2,1.000,0,9,9,3,1,2,4,0,24,8,1,0,LAC
3,22023,201142,0022301156,"APR 09, 2024",PHX vs. LAC,L,40,8,22,0.364,2,5,0.400,3,5,0.600,0,2,2,2,2,0,1,2,21,-14,1,1,LAC
4,22023,201142,0022301138,"APR 07, 2024",PHX vs. NOP,L,38,8,22,0.364,3,8,0.375,4,4,1.000,0,4,4,6,0,0,0,1,23,-12,1,1,NOP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,22022,201142,0022200066,"OCT 27, 2022",BKN vs. DAL,L,41,12,21,0.571,2,5,0.400,11,11,1.000,1,2,3,5,2,1,3,2,37,-6,1,1,DAL
43,22022,201142,0022200060,"OCT 26, 2022",BKN @ MIL,L,39,10,23,0.435,1,4,0.250,12,13,0.923,0,6,6,3,0,2,5,3,33,-17,1,0,MIL
44,22022,201142,0022200049,"OCT 24, 2022",BKN @ MEM,L,39,14,20,0.700,1,3,0.333,8,9,0.889,1,4,5,4,0,0,1,4,37,-11,1,0,MEM
45,22022,201142,0022200021,"OCT 21, 2022",BKN vs. TOR,W,35,8,18,0.444,3,9,0.333,8,10,0.800,1,4,5,6,0,2,5,2,27,-8,1,1,TOR


In [25]:
X, y = df.drop(columns=["SEASON_ID", "Player_ID", "Game_ID", "GAME_DATE", "VIDEO_AVAILABLE", "MATCHUP", "OPPONENT", "WL"]), df["PTS"]
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = .2, random_state=1)

scaler = StandardScaler()

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=2)
X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.fit_transform(X_val.values)
alphavec = 10**np.linspace(-2,2,200)

lasso_cv = LassoCV(alphas = alphavec, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

C:\Users\129772\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.877e-01, tolerance: 3.705e-01
  model = cd_fast.enet_coordinate_descent(


LassoCV(alphas=array([1.00000000e-02, 1.04737090e-02, 1.09698580e-02, 1.14895100e-02,
       1.20337784e-02, 1.26038293e-02, 1.32008840e-02, 1.38262217e-02,
       1.44811823e-02, 1.51671689e-02, 1.58856513e-02, 1.66381689e-02,
       1.74263339e-02, 1.82518349e-02, 1.91164408e-02, 2.00220037e-02,
       2.09704640e-02, 2.19638537e-02, 2.30043012e-02, 2.40940356e-02,
       2.52353917e-02, 2.64308149e-0...
       3.44896226e+01, 3.61234270e+01, 3.78346262e+01, 3.96268864e+01,
       4.15040476e+01, 4.34701316e+01, 4.55293507e+01, 4.76861170e+01,
       4.99450512e+01, 5.23109931e+01, 5.47890118e+01, 5.73844165e+01,
       6.01027678e+01, 6.29498899e+01, 6.59318827e+01, 6.90551352e+01,
       7.23263390e+01, 7.57525026e+01, 7.93409667e+01, 8.30994195e+01,
       8.70359136e+01, 9.11588830e+01, 9.54771611e+01, 1.00000000e+02]),
        cv=5)

In [26]:
for col, coef in zip(X_train.columns, lasso_cv.coef_):
    print(f"{col:<16}: {coef:>12,.7f}")

MIN             :    0.0000000
FGM             :    1.9224144
FGA             :    0.0000000
FG_PCT          :    0.0000000
FG3M            :    0.3877443
FG3A            :    0.0000000
FG3_PCT         :    0.0000000
FTM             :    1.1140013
FTA             :    0.0000000
FT_PCT          :    0.0000000
OREB            :   -0.0000000
DREB            :    0.0000000
REB             :   -0.0000000
AST             :   -0.0000000
STL             :   -0.0000000
BLK             :    0.0000000
TOV             :    0.0000000
PF              :    0.0000000
PTS             :    4.8284523
PLUS_MINUS      :   -0.0000000
HOME            :    0.0000000
